In [ ]:
# !git clone https://github.com/ultralytics/ultralytics
# !pip install ultralytics
# cd ultralytics
# !pip install -r requirements.txt
# cd ..

In [ ]:
# # YOLOv8n 모델 다시 다운로드
# !wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

In [ ]:
import torch

print(torch.cuda.memory_summary())
torch.cuda.empty_cache()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [1]:
import torch

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 가능 여부: {torch.cuda.is_available()}")
print(f"현재 사용 중인 CUDA 버전: {torch.version.cuda}")
print(f"CUDNN 버전: {torch.backends.cudnn.version()}")

PyTorch 버전: 2.5.1+cu124
CUDA 가능 여부: True
현재 사용 중인 CUDA 버전: 12.4
CUDNN 버전: 90100


In [2]:
!nvidia-smi

Wed Nov 20 06:49:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  | 00000000:A1:00.0 Off |                    0 |
| N/A   36C    P0              43W / 250W |     16MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import torch
print(torch.cuda.is_available())  # True가 출력되어야 GPU 사용 가능

True


- 라벨링 데이터가 골프공 하나일때만 돌리기!!

In [6]:
import os
dir = ['train', 'valid', 'test']

def change_class_number(label_dir, new_class_number):
    # 디렉토리 내 모든 .txt 파일을 순회
    label_dir = f'./merged_golfball/{label_dir}/labels'
    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(label_dir, label_file)

            # 라벨 파일 읽기
            with open(label_path, 'r') as file:
                lines = file.readlines()

            # 각 줄의 클래스 번호를 new_class_number로 변경
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    parts[0] = str(new_class_number)  # 클래스 번호를 new_class_number로 변경
                    new_lines.append(' '.join(parts))

            # 수정된 내용을 다시 라벨 파일에 쓰기
            with open(label_path, 'w') as file:
                file.write('\n'.join(new_lines) + '\n')

    print(f"라벨링 번호가 모두 {new_class_number}로 변경되었습니다.")

for label_dir in dir:
    change_class_number(label_dir, 0)

라벨링 번호가 모두 0로 변경되었습니다.
라벨링 번호가 모두 0로 변경되었습니다.
라벨링 번호가 모두 0로 변경되었습니다.


## 모델학습

- 새 데이터 학습

In [ ]:
# # 학습전 설치 필수
# sudo apt-get update
# sudo apt-get install libgl1-mesa-glx

##### 이미지 전처리 및 대비 향상
- 화질이 낮은 이미지의 세부 정보와 대비를 개선하기 위해 전처리 기술을 사용하는 것이 중요합니다. 예를 들어:
	- Entropy와 Fidelity 기반 대비 조정: 저조도 이미지에서 세부 정보를 명확히 하고 노이즈를 줄이는 데 효과적입니다. 적응형 2차원 위너 필터를 활용하면 객체의 형상을 유지하면서 노이즈를 제거할 수 있습니다.
	- 히스토그램 평활화 또는 Retinex 기반 기법: 픽셀 간 대비를 향상시켜 객체 탐지 성능을 높이는 데 유용합니다.

In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
import yaml
from PIL import Image

# 사용자 정의 전처리 함수 정의
def preprocess_image(image_path):
    """
    이미지 전처리 함수: 히스토그램 평활화와 적응형 위너 필터 적용.
    Args:
        image_path (str): 원본 이미지 경로.
    Returns:
        preprocessed_image (np.ndarray): 전처리된 이미지.
    """
    # 이미지 읽기 (컬러)
    image = Image.open(image_path)
    # 전처리 작업 수행 (예: 크기 조정, 정규화 등)
    return image

# 데이터셋 디렉토리에서 모든 이미지를 전처리하는 함수
def preprocess_dataset():
    """
    데이터셋 디렉토리의 모든 이미지를 전처리하여 저장
    """
    
    # 학습 및 검증 이미지 경로 설정
    train_images_dir = Path('./GolfBall_data_for_tracking/train/images')
    val_images_dir = Path('./GolfBall_data_for_tracking/valid/images')
    
    # 전처리된 이미지 저장 경로 설정
    train_output_dir = Path('./GolfBall_data_for_tracking/train/images_preprocessed')
    val_output_dir = Path('./GolfBall_data_for_tracking/valid/images_preprocessed')

    # 출력 디렉토리 생성
    train_output_dir.mkdir(parents=True, exist_ok=True)
    val_output_dir.mkdir(parents=True, exist_ok=True)

    # 학습 데이터 전처리
    for image_path in train_images_dir.glob('*.jpg'):  # 모든 JPG 파일 처리
        preprocessed_image = preprocess_image(str(image_path))
        output_path = train_output_dir / image_path.name
        preprocessed_image.save(output_path)

    # 검증 데이터 전처리
    for image_path in val_images_dir.glob('*.jpg'):  # 모든 JPG 파일 처리
        preprocessed_image = preprocess_image(str(image_path))
        output_path = val_output_dir / image_path.name
        preprocessed_image.save(output_path)

    print("모든 이미지의 전처리가 완료되었습니다.")

# 전처리 실행
preprocess_dataset()

모든 이미지의 전처리가 완료되었습니다.


In [ ]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./final_split_golfball_dataset/data.yaml',  # YAML 파일 경로
    epochs=500, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golf_ball_final_merged_500epoch_16batch_10patience',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 x활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

    479/500      2.39G     0.5759     0.2978     0.8605          1        640: 100%|██████████| 1137/1137 [02:21<00:00,  8.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:20<00:00,  3.22it/s]

                   all       2138       2536      0.979      0.985      0.992      0.852



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    480/500      2.26G     0.5684     0.2938     0.8601          0        640: 100%|██████████| 1137/1137 [02:20<00:00,  8.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:20<00:00,  3.20it/s]

                   all       2138       2536      0.978      0.985      0.992      0.853


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    482/500      2.37G     0.5671     0.2939     0.8583          0        640: 100%|██████████| 1137/1137 [02:24<00:00,  7.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:15<00:00,  4.28it/s]

                   all       2138       2536      0.979      0.984      0.992      0.854



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    483/500      2.33G     0.5646     0.2918     0.8603         21        640:  55%|█████▌    | 629/1137 [01:21<01:02,  8.13it/s]

In [8]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./GolfBall_data_for_tracking/data.yaml',  # YAML 파일 경로
    epochs=300, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golf_ball_new_300epoch_16batch_10patience2',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 x활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

New https://pypi.org/project/ultralytics/8.3.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40339MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./GolfBall_data_for_tracking/data.yaml, epochs=300, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_golf_ball_new_300epoch_16batch_10patience22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, c

Model summary: 225 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/yolov8n_golf_ball_new_300epoch_16batch_10patience22', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /root/YOLOv8n_golf_train/GolfBall_data_for_tracking/train/labels... 13944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13944/13944 [00:14<00:00, 989.05it/s] 


train: New cache created: /root/YOLOv8n_golf_train/GolfBall_data_for_tracking/train/labels.cache


val: Scanning /root/YOLOv8n_golf_train/GolfBall_data_for_tracking/valid/labels... 3488 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3488/3488 [00:03<00:00, 964.35it/s]


...

                   all       3488       3488      0.989      0.976      0.994      0.811



300 epochs completed in 6.618 hours.
Optimizer stripped from runs/detect/yolov8n_golf_ball_new_300epoch_16batch_10patience22/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/yolov8n_golf_ball_new_300epoch_16batch_10patience22/weights/best.pt, 6.3MB

Validating runs/detect/yolov8n_golf_ball_new_300epoch_16batch_10patience22/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40339MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:14<00:00,  7.33it/s]


                   all       3488       3488      0.977      0.979      0.993      0.773
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/yolov8n_golf_ball_new_300epoch_16batch_10patience22


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f129c2fd700>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./GolfBall_data_for_tracking/data.yaml',  # YAML 파일 경로
    epochs=500, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golf_ball_new_merged_500epoch_16batch_10patience',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 x활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

New https://pypi.org/project/ultralytics/8.3.35 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40339MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./GolfBall_data_for_tracking/data.yaml, epochs=500, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_golf_ball_new_500epoch_16batch_10patience, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, cla

train: Scanning /root/YOLOv8n_golf_train/GolfBall_data_for_tracking/train/labels.cache... 13944 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13944/13944 [00:00<?, ?it/s]
val: Scanning /root/YOLOv8n_golf_train/GolfBall_data_for_tracking/valid/labels.cache... 3488 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3488/3488 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8n_golf_ball_new_500epoch_16batch_10patience/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/yolov8n_golf_ball_new_500epoch_16batch_10patience
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/500      2.48G      1.967      2.671      1.106          7        640: 100%|██████████| 872/872 [01:12<00:00, 12.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:11<00:00,  9.47it/s]


                   all       3488       3488      0.504      0.755      0.592      0.251

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       3488       3488      0.992      0.993      0.995      0.867



500 epochs completed in 10.245 hours.
Optimizer stripped from runs/detect/yolov8n_golf_ball_new_500epoch_16batch_10patience/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/yolov8n_golf_ball_new_500epoch_16batch_10patience/weights/best.pt, 6.3MB

Validating runs/detect/yolov8n_golf_ball_new_500epoch_16batch_10patience/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40339MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 109/109 [00:14<00:00,  7.35it/s]


                   all       3488       3488      0.986      0.989      0.994      0.824
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/yolov8n_golf_ball_new_500epoch_16batch_10patience


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f12f5169f70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

- 골프공만 학습

In [ ]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./merged_golfball/data.yaml',  # YAML 파일 경로
    epochs=100, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golf_ball_merged_100epoch_16batch_10patience',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

- 골프공, 골프채(헤드, 핸들) 모두 학습

In [ ]:
from ultralytics import YOLO

# YOLOv8n 모델 로드 (사전학습된 모델)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    data='./merged_golfball_with_golf_club/data.yaml',  # YAML 파일 경로
    epochs=200, 
    imgsz=640, # 이미지 크기
    batch=16,
    name='yolov8n_golf_ball_merged_golfball_with_golf_club_200epoch_16batch_10patience',
    patience=10,  # 10 epoch 동안 개선이 없으면 조기 종료
    augment=True  # 데이터 증강 활성화, 여러 증강 기법들이 학습 과정에서 자동으로 적용, 여기에는 회전, 크기 조정, 좌우 반전, 밝기 및 대비 조정 등의 다양한 변환이 포함
)

# with open('output.txt', 'w') as f:
#     f.write(str(output_data))

New https://pypi.org/project/ultralytics/8.3.29 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.27 🚀 Python-3.9.20 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24111MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./merged_golfball_with_golf_club/data.yaml, epochs=200, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_golf_ball_merged_golfball_with_golf_club_150epoch_16batch_10patience, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augm

train: Scanning /root/YOLOv8n_golf_train/merged_golfball_with_golf_club/train/labels... 24867 images, 288 backgrounds, 0 corrupt: 100%|██████████| 24867/24867 [00:19<00:00, 1298.52it/s]


train: New cache created: /root/YOLOv8n_golf_train/merged_golfball_with_golf_club/train/labels.cache


val: Scanning /root/YOLOv8n_golf_train/merged_golfball_with_golf_club/valid/labels... 1904 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1904/1904 [00:01<00:00, 1229.39it/s]

val: New cache created: /root/YOLOv8n_golf_train/merged_golfball_with_golf_club/valid/labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    138/200      2.46G      1.275     0.9197      1.003         12        640: 100%|██████████| 1555/1555 [01:45<00:00, 14.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 60/60 [00:05<00:00, 10.29it/s]

                   all       1904       4236      0.818      0.829        0.8      0.502



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    139/200      2.47G      1.276      0.917     0.9993          3        640: 100%|██████████| 1555/1555 [01:57<00:00, 13.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  17%|█▋        | 10/60 [00:01<00:05,  9.53it/s]

In [1]:
# 검증 데이터셋으로 모델 평가
results = model.val()

NameError: name 'model' is not defined

In [7]:
import pandas as pd
results_df = pd.read_csv('/root/YOLO_GOLF/runs/detect/yolov8n_golf_ball_100epoch_10patience_8batch/results.csv')
results_df

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,17.7974,1.01952,1.53185,1.20647,0.80259,0.77491,0.79671,0.49691,1.16135,1.41043,1.37551,0.000664,0.000664,0.000664
1,2,31.7419,1.07298,1.17419,1.23323,0.85813,0.76864,0.82944,0.57584,1.04713,1.29217,1.25514,0.001318,0.001318,0.001318
2,3,45.4636,1.06847,1.02328,1.23297,0.84013,0.79834,0.83440,0.61019,0.95966,0.88772,1.19697,0.001958,0.001958,0.001958
3,4,59.0101,1.04322,0.94504,1.22133,0.89720,0.82873,0.84483,0.62366,0.98856,0.80684,1.18422,0.001941,0.001941,0.001941
4,5,72.5478,1.01203,0.88575,1.19647,0.87866,0.82597,0.86330,0.60810,1.00340,0.73188,1.20598,0.001921,0.001921,0.001921
5,6,85.9300,0.99668,0.85178,1.19100,0.88926,0.84293,0.88276,0.66772,0.93437,0.65269,1.16236,0.001901,0.001901,0.001901
6,7,99.3824,0.98235,0.77483,1.18446,0.88562,0.81281,0.86249,0.63525,0.94674,0.78779,1.15482,0.001881,0.001881,0.001881
7,8,112.5970,0.95683,0.77778,1.16248,0.89974,0.81806,0.88123,0.66109,0.87685,0.70493,1.15583,0.001861,0.001861,0.001861
8,9,126.0160,0.92254,0.74623,1.14646,0.94185,0.85635,0.90279,0.68057,0.90936,0.59504,1.13064,0.001842,0.001842,0.001842
9,10,139.4550,0.92967,0.73042,1.14381,0.89874,0.85810,0.89501,0.68420,0.86295,0.59679,1.10322,0.001822,0.001822,0.001822


In [6]:
# 이미지에 대한 예측 수행
results = model.predict('/root/YOLO_GOLF/GolfBallDetector-10/test/images/6ced8d90b264f0ac_jpg.rf.d6f2a323158e8412ae30ee43542f1a4a.jpg', conf=0.25)
results


image 1/1 /root/YOLO_GOLF/GolfBallDetector-10/test/images/6ced8d90b264f0ac_jpg.rf.d6f2a323158e8412ae30ee43542f1a4a.jpg: 640x640 2 golfballs, 82.4ms
Speed: 1.8ms preprocess, 82.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'golfball'}
 obb: None
 orig_img: array([[[201, 171, 144],
         [202, 173, 146],
         [204, 175, 148],
         ...,
         [186, 164, 129],
         [186, 164, 129],
         [184, 162, 127]],
 
        [[206, 176, 151],
         [207, 178, 151],
         [207, 180, 154],
         ...,
         [184, 162, 127],
         [186, 161, 127],
         [185, 163, 128]],
 
        [[211, 183, 159],
         [211, 186, 160],
         [212, 186, 162],
         ...,
         [183, 159, 123],
         [187, 161, 125],
         [188, 164, 128]],
 
        ...,
 
        [[ 33,  47,  45],
         [ 34,  48,  46],
         [ 37,  51,  49],
         ...,
         [ 22,  79,  58],
         [ 21,  78,  57],
         [ 21,  78,  57]],
 
        [[ 29,  43,  41],
         [ 29,  43,  41],
         [ 29,  43,  41],
         ...,
         [ 24,  